In [3]:
import arcpy
from arcpy import env
env.overwriteOutput = True
env.workspace = r"C:\ArcGisScripts"


In [ ]:
arcpy.management.CreateFileGDB(
    out_folder_path="tmp",
    out_name="geodb",
    out_version="CURRENT"
)

In [2]:
arcpy.management.XYTableToPoint(
    in_table=r"C:\ArcgisScripts\data\predictions_7days_20230523_to_20230530.csv",
    out_feature_class=r"tmp\geodb.gdb\prediction_points",
    x_field="longitude",
    y_field="latitude",
    z_field=None,
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
)

<Result 'tmp\\geodb.gdb\\prediction_points'>

In [6]:
from datetime import date, timedelta

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [7]:
start_date = date(2023, 5, 23)
end_date = date(2023, 5, 30)
date_range = int((end_date - start_date).days);

rasterTimes = []
for single_date in daterange(start_date, end_date):
    rasterTimes += ["timestamp '"+single_date.strftime("%Y/%m/%d")+" 12:00:00'"]

rasterNames = []
for single_date in daterange(start_date, end_date):
    rasterNames += [single_date.strftime("%d_%m_%Y")]
    
species = ['Torsk','Hyse']
gears = ['Andre liner', 'Bunntrål', 'Settegarn', 'Snurrevad', 'Udefinert garn']

#species = ['Torsk']
#gears = ['Andre liner', 'Bunntrål']

print(rasterTimes, rasterNames)
print(species, gears)

def rn(s,g,n):
    return s+"_"+g.replace(" ","_")+"_"+n

["timestamp '2023/05/23 12:00:00'", "timestamp '2023/05/24 12:00:00'", "timestamp '2023/05/25 12:00:00'", "timestamp '2023/05/26 12:00:00'", "timestamp '2023/05/27 12:00:00'", "timestamp '2023/05/28 12:00:00'", "timestamp '2023/05/29 12:00:00'"] ['23_05_2023', '24_05_2023', '25_05_2023', '26_05_2023', '27_05_2023', '28_05_2023', '29_05_2023']
['Torsk', 'Hyse'] ['Andre liner', 'Bunntrål', 'Settegarn', 'Snurrevad', 'Udefinert garn']


In [5]:
for count, rasterTime in enumerate(rasterTimes, start=0):
    for g in gears:
        for s in species:
            rd = rn(s,g,rasterNames[count])
            arcpy.MakeFeatureLayer_management(r"tmp\geodb.gdb\prediction_points", "Points_Layer", "gear = '"+g+"' And species = '"+s+"' And time = "+rasterTime)
            arcpy.conversion.PointToRaster(
                in_features="Points_Layer",
                value_field="predictions",
                out_rasterdataset=r"tmp\geodb.gdb\Predictions_"+rd,
                cell_assignment="MOST_FREQUENT",
                priority_field="NONE",
                cellsize=0.1,
                build_rat="BUILD"
            )
            print("Predictions_"+rd)
            time.sleep(0.1)

Predictions_Torsk_Andre_liner_23_05_2023
Predictions_Hyse_Andre_liner_23_05_2023
Predictions_Torsk_Bunntrål_23_05_2023
Predictions_Hyse_Bunntrål_23_05_2023
Predictions_Torsk_Settegarn_23_05_2023
Predictions_Hyse_Settegarn_23_05_2023
Predictions_Torsk_Snurrevad_23_05_2023
Predictions_Hyse_Snurrevad_23_05_2023
Predictions_Torsk_Udefinert_garn_23_05_2023
Predictions_Hyse_Udefinert_garn_23_05_2023
Predictions_Torsk_Andre_liner_24_05_2023
Predictions_Hyse_Andre_liner_24_05_2023
Predictions_Torsk_Bunntrål_24_05_2023
Predictions_Hyse_Bunntrål_24_05_2023
Predictions_Torsk_Settegarn_24_05_2023
Predictions_Hyse_Settegarn_24_05_2023
Predictions_Torsk_Snurrevad_24_05_2023
Predictions_Hyse_Snurrevad_24_05_2023
Predictions_Torsk_Udefinert_garn_24_05_2023
Predictions_Hyse_Udefinert_garn_24_05_2023
Predictions_Torsk_Andre_liner_25_05_2023
Predictions_Hyse_Andre_liner_25_05_2023
Predictions_Torsk_Bunntrål_25_05_2023
Predictions_Hyse_Bunntrål_25_05_2023
Predictions_Torsk_Settegarn_25_05_2023
Predictions

In [6]:
for rasterName in rasterNames:
    for g in gears:
        for s in species:
            rd = rn(s,g,rasterName)
            arcpy.ddd.Contour(
                in_raster=r"tmp\geodb.gdb\Predictions_"+rd,
                out_polyline_features=r"tmp\geodb.gdb\Contours_"+rd,
                contour_interval=500,
                base_contour=0,
                z_factor=1,
                contour_type="CONTOUR_POLYGON",
                max_vertices_per_feature=None
            )
            print("Contours_"+rd)
            time.sleep(0.1)

Contours_Torsk_Andre_liner_23_05_2023
Contours_Hyse_Andre_liner_23_05_2023
Contours_Torsk_Bunntrål_23_05_2023
Contours_Hyse_Bunntrål_23_05_2023
Contours_Torsk_Settegarn_23_05_2023
Contours_Hyse_Settegarn_23_05_2023
Contours_Torsk_Snurrevad_23_05_2023
Contours_Hyse_Snurrevad_23_05_2023
Contours_Torsk_Udefinert_garn_23_05_2023
Contours_Hyse_Udefinert_garn_23_05_2023
Contours_Torsk_Andre_liner_24_05_2023
Contours_Hyse_Andre_liner_24_05_2023
Contours_Torsk_Bunntrål_24_05_2023
Contours_Hyse_Bunntrål_24_05_2023
Contours_Torsk_Settegarn_24_05_2023
Contours_Hyse_Settegarn_24_05_2023
Contours_Torsk_Snurrevad_24_05_2023
Contours_Hyse_Snurrevad_24_05_2023
Contours_Torsk_Udefinert_garn_24_05_2023
Contours_Hyse_Udefinert_garn_24_05_2023
Contours_Torsk_Andre_liner_25_05_2023
Contours_Hyse_Andre_liner_25_05_2023
Contours_Torsk_Bunntrål_25_05_2023
Contours_Hyse_Bunntrål_25_05_2023
Contours_Torsk_Settegarn_25_05_2023
Contours_Hyse_Settegarn_25_05_2023
Contours_Torsk_Snurrevad_25_05_2023
Contours_Hyse_S

In [7]:
for rasterName in rasterNames:
    for g in gears:
        for s in species:
            rd = rn(s,g,rasterName)
            arcpy.cartography.SmoothPolygon(
                in_features=r"tmp\geodb.gdb\Contours_"+rd,
                out_feature_class=r"tmp\geodb.gdb\Smooth_"+rd,
                algorithm="PAEK",
                tolerance="100 Kilometers",
                endpoint_option="FIXED_ENDPOINT",
                error_option="NO_CHECK",
                in_barriers=None
            )
            print("Smooth_"+rd)
            time.sleep(0.1)

Smooth_Torsk_Andre_liner_23_05_2023
Smooth_Hyse_Andre_liner_23_05_2023
Smooth_Torsk_Bunntrål_23_05_2023
Smooth_Hyse_Bunntrål_23_05_2023
Smooth_Torsk_Settegarn_23_05_2023
Smooth_Hyse_Settegarn_23_05_2023
Smooth_Torsk_Snurrevad_23_05_2023
Smooth_Hyse_Snurrevad_23_05_2023
Smooth_Torsk_Udefinert_garn_23_05_2023
Smooth_Hyse_Udefinert_garn_23_05_2023
Smooth_Torsk_Andre_liner_24_05_2023
Smooth_Hyse_Andre_liner_24_05_2023
Smooth_Torsk_Bunntrål_24_05_2023
Smooth_Hyse_Bunntrål_24_05_2023
Smooth_Torsk_Settegarn_24_05_2023
Smooth_Hyse_Settegarn_24_05_2023
Smooth_Torsk_Snurrevad_24_05_2023
Smooth_Hyse_Snurrevad_24_05_2023
Smooth_Torsk_Udefinert_garn_24_05_2023
Smooth_Hyse_Udefinert_garn_24_05_2023
Smooth_Torsk_Andre_liner_25_05_2023
Smooth_Hyse_Andre_liner_25_05_2023
Smooth_Torsk_Bunntrål_25_05_2023
Smooth_Hyse_Bunntrål_25_05_2023
Smooth_Torsk_Settegarn_25_05_2023
Smooth_Hyse_Settegarn_25_05_2023
Smooth_Torsk_Snurrevad_25_05_2023
Smooth_Hyse_Snurrevad_25_05_2023
Smooth_Torsk_Udefinert_garn_25_05_20

In [8]:
for rasterName in rasterNames:
    for g in gears:
        for s in species:
            rd = rn(s,g,rasterName)
            table = r"tmp\geodb.gdb\Smooth_"+rd
            arcpy.management.AddField(
                in_table=table,
                field_name="Date",
                field_type="DATE",
                field_precision=None,
                field_scale=None,
                field_length=None,
                field_alias="Date",
                field_is_nullable="NULLABLE",
                field_is_required="NON_REQUIRED",
                field_domain=""
            )
            time.sleep(0.1)
            arcpy.management.CalculateField(
                in_table=table,
                field="Date",
                expression='datetime.datetime.strptime("'+rasterName+'", "%d_%m_%Y")',
                expression_type="PYTHON3",
                code_block="",
                field_type="TEXT",
                enforce_domains="NO_ENFORCE_DOMAINS"
            )
            time.sleep(0.1)
            arcpy.management.AddField(
                in_table=table,
                field_name="FAOSpecies",
                field_type="TEXT",
                field_precision=None,
                field_scale=None,
                field_length=None,
                field_alias="FAOSpecies",
                field_is_nullable="NULLABLE",
                field_is_required="NON_REQUIRED",
                field_domain=""
            )
            time.sleep(0.1)
            arcpy.management.AddField(
                in_table=table,
                field_name="FDIRGear",
                field_type="TEXT",
                field_precision=None,
                field_scale=None,
                field_length=None,
                field_alias="FDIRGear",
                field_is_nullable="NULLABLE",
                field_is_required="NON_REQUIRED",
                field_domain=""
            )
            time.sleep(0.1)
            arcpy.management.CalculateField(
                in_table=table,
                field="FAOSpecies",
                expression='"'+s+'"',
                expression_type="PYTHON3",
                code_block="",
                field_type="TEXT",
                enforce_domains="NO_ENFORCE_DOMAINS"
            )
            time.sleep(0.1)
            arcpy.management.CalculateField(
                in_table=table,
                field="FDIRGear",
                expression='"'+g+'"',
                expression_type="PYTHON3",
                code_block="",
                field_type="TEXT",
                enforce_domains="NO_ENFORCE_DOMAINS"
            )
            time.sleep(0.1)
            print(table)

tmp\geodb.gdb\Smooth_Torsk_Andre_liner_23_05_2023
tmp\geodb.gdb\Smooth_Hyse_Andre_liner_23_05_2023
tmp\geodb.gdb\Smooth_Torsk_Bunntrål_23_05_2023
tmp\geodb.gdb\Smooth_Hyse_Bunntrål_23_05_2023
tmp\geodb.gdb\Smooth_Torsk_Settegarn_23_05_2023
tmp\geodb.gdb\Smooth_Hyse_Settegarn_23_05_2023
tmp\geodb.gdb\Smooth_Torsk_Snurrevad_23_05_2023
tmp\geodb.gdb\Smooth_Hyse_Snurrevad_23_05_2023
tmp\geodb.gdb\Smooth_Torsk_Udefinert_garn_23_05_2023
tmp\geodb.gdb\Smooth_Hyse_Udefinert_garn_23_05_2023
tmp\geodb.gdb\Smooth_Torsk_Andre_liner_24_05_2023
tmp\geodb.gdb\Smooth_Hyse_Andre_liner_24_05_2023
tmp\geodb.gdb\Smooth_Torsk_Bunntrål_24_05_2023
tmp\geodb.gdb\Smooth_Hyse_Bunntrål_24_05_2023
tmp\geodb.gdb\Smooth_Torsk_Settegarn_24_05_2023
tmp\geodb.gdb\Smooth_Hyse_Settegarn_24_05_2023
tmp\geodb.gdb\Smooth_Torsk_Snurrevad_24_05_2023
tmp\geodb.gdb\Smooth_Hyse_Snurrevad_24_05_2023
tmp\geodb.gdb\Smooth_Torsk_Udefinert_garn_24_05_2023
tmp\geodb.gdb\Smooth_Hyse_Udefinert_garn_24_05_2023
tmp\geodb.gdb\Smooth_Torsk

In [8]:
appendTables = []
for count, rasterTime in enumerate(rasterTimes, start=0):
    for gear in gears:
        for s in species:
            appendTables += [r"tmp\geodb.gdb\Smooth_"+s+"_"+gear.replace(" ","_")+"_"+rasterNames[count]]

rootTable = appendTables[0]
            
appendTables.pop(0)
appendInputTables = ';'.join(appendTables)
print("root: "+rootTable)
print("append: "+appendInputTables)

root: tmp\geodb.gdb\Smooth_Torsk_Andre_liner_23_05_2023
append: tmp\geodb.gdb\Smooth_Hyse_Andre_liner_23_05_2023;tmp\geodb.gdb\Smooth_Torsk_Bunntrål_23_05_2023;tmp\geodb.gdb\Smooth_Hyse_Bunntrål_23_05_2023;tmp\geodb.gdb\Smooth_Torsk_Settegarn_23_05_2023;tmp\geodb.gdb\Smooth_Hyse_Settegarn_23_05_2023;tmp\geodb.gdb\Smooth_Torsk_Snurrevad_23_05_2023;tmp\geodb.gdb\Smooth_Hyse_Snurrevad_23_05_2023;tmp\geodb.gdb\Smooth_Torsk_Udefinert_garn_23_05_2023;tmp\geodb.gdb\Smooth_Hyse_Udefinert_garn_23_05_2023;tmp\geodb.gdb\Smooth_Torsk_Andre_liner_24_05_2023;tmp\geodb.gdb\Smooth_Hyse_Andre_liner_24_05_2023;tmp\geodb.gdb\Smooth_Torsk_Bunntrål_24_05_2023;tmp\geodb.gdb\Smooth_Hyse_Bunntrål_24_05_2023;tmp\geodb.gdb\Smooth_Torsk_Settegarn_24_05_2023;tmp\geodb.gdb\Smooth_Hyse_Settegarn_24_05_2023;tmp\geodb.gdb\Smooth_Torsk_Snurrevad_24_05_2023;tmp\geodb.gdb\Smooth_Hyse_Snurrevad_24_05_2023;tmp\geodb.gdb\Smooth_Torsk_Udefinert_garn_24_05_2023;tmp\geodb.gdb\Smooth_Hyse_Udefinert_garn_24_05_2023;tmp\geodb.gd

In [10]:
arcpy.management.Append(
    inputs=appendInputTables,
    target=rootTable,
    schema_type="TEST",
    field_mapping=None,
    subtype="",
    expression="",
    match_fields=None,
    update_geometry="NOT_UPDATE_GEOMETRY"
)
print("append done")

append done


In [12]:
arcpy.management.AlterField(
    in_table=rootTable,
    field="ContourMin",
    new_field_name="prediction",
    new_field_alias="prediction",
    field_type="DOUBLE",
    field_length=8,
    clear_field_alias="DO_NOT_CLEAR"
)
time.sleep(0.1)
arcpy.management.DeleteField(
    in_table=rootTable,
    drop_field="ContourMax",
    method="DELETE_FIELDS"
)

<Result 'tmp\\geodb.gdb\\Smooth_Torsk_Andre_liner_23_05_2023'>

In [9]:
# Print fields in table

fieldObjects = arcpy.ListFields(rootTable)
for field in fieldObjects:
    print(field.name)

OBJECTID
Shape
prediction
Shape_Length
Shape_Area
InPoly_FID
Date
FAOSpecies
FDIRGear


In [10]:
# Print 100 random rows

import random

n = arcpy.GetCount_management(rootTable)
cursor = arcpy.SearchCursor(rootTable)
c = 0
for count, row in enumerate(cursor,start=1):
    if c < 100:
        if random.random() > 0.8:
            c = c+1
            print("[{0}:{1}]\t\t{2}\t\t{3}\t\t{4}\t{5}".format(count, n, row.FAOSpecies, row.FDIRGear, str(row.Date).split(" ")[0], row.prediction))

[6:1118]		Torsk		Andre liner		2023-05-23	2500.0
[7:1118]		Torsk		Andre liner		2023-05-23	3000.0
[13:1118]		Torsk		Andre liner		2023-05-23	6000.0
[21:1118]		Torsk		Andre liner		2023-05-23	10000.0
[22:1118]		Torsk		Andre liner		2023-05-23	10500.0
[26:1118]		Torsk		Andre liner		2023-05-23	12500.0
[32:1118]		Hyse		Andre liner		2023-05-23	2500.0
[46:1118]		Torsk		Bunntrål		2023-05-23	5500.0
[62:1118]		Torsk		Bunntrål		2023-05-23	13500.0
[66:1118]		Hyse		Bunntrål		2023-05-23	1500.0
[69:1118]		Hyse		Bunntrål		2023-05-23	3000.0
[70:1118]		Hyse		Bunntrål		2023-05-23	3500.0
[79:1118]		Torsk		Settegarn		2023-05-23	3000.0
[86:1118]		Torsk		Settegarn		2023-05-23	6500.0
[90:1118]		Torsk		Settegarn		2023-05-23	8500.0
[96:1118]		Hyse		Settegarn		2023-05-23	46.0
[100:1118]		Hyse		Settegarn		2023-05-23	2000.0
[106:1118]		Torsk		Snurrevad		2023-05-23	2500.0
[107:1118]		Torsk		Snurrevad		2023-05-23	3000.0
[127:1118]		Hyse		Snurrevad		2023-05-23	108.0
[132:1118]		Hyse		Snurrevad		2023-05-23	2500.0
[136:111

In [11]:
import os

export_folder = r"C:/ArcgisScripts/tmp/export"

if not os.path.isdir(export_folder):
    os.mkdir(export_folder)

arcpy.conversion.FeatureClassToShapefile(
    Input_Features=rootTable,
    Output_Folder=export_folder
)

<Result 'C:\\ArcgisScripts\\tmp\\export'>

In [15]:
import shutil

zip_path = r"C:/ArcgisScripts/tmp/pred"
shutil.make_archive(zip_path, "zip", export_folder)

'C:\\ArcgisScripts\\tmp\\pred.zip'

In [18]:
import requests

# Step 1: Set up the necessary variables
api_key = "AAPK3143ee20e5ef48e8b3645ca1128c1b053oBpM1V58j4tyMXCu_Ba3jVzs_S3CCAlyQF6Jx_i5DorcKAcx_X6b2IcD5_PGnDl"
username = "post4k_dev"
shapefile_zip_path = zip_path+".zip"

upload_url = f"https://www.arcgis.com/sharing/rest/content/users/{username}/addItem"

headers = {
    "Authorization": f"Bearer {api_key}",
}

files = {
    "file": open(shapefile_zip_path, "rb")
}

upload_response = requests.post(upload_url, headers=headers, files=files)

try:
    upload_response.raise_for_status()  # Raise an exception if the request was not successful
    upload_json = upload_response.json()
    item_id = upload_json["id"]
    print("Shapefile uploaded successfully.")
except Exception as e:
    print("Error occurred while uploading the shapefile.")
    print("Response:", upload_response.text)
    raise e

Error occurred while uploading the shapefile.
Response: <!DOCTYPE html>





<html>
<head>
<title>ArcGIS Portal Directory</title>
<link href="/sharing/rest/files/gw.css" rel="stylesheet" type="text/css"/>
</head>
<body>
<br/>
<div class="gwDiv">
<table class="navTable" width="100%"><tr><td class="breadcrumbs">ArcGIS Portal Directory</td></tr></table>
<tr valign="top">
<br/><br/>
<b>
<a href="http://resources.arcgis.com/en/help/arcgis-rest-api/" >API Reference</a>
</b>

</tr>
</table>

</div>

</body>

</html>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# Step 3: Publish the shapefile as a hosted feature layer
publish_url = f"https://www.arcgis.com/sharing/rest/content/users/{username}/publish"

publish_params = {
    "name": "Pred layer",
    "filetype": "shapefile",
    "itemId": item_id,
    "publishParameters": {
        "layerInfo": {
            "name": "pred layer"
        }
    }
}

publish_response = requests.post(publish_url, headers=headers, json=publish_params)
publish_json = publish_response.json()
hosted_feature_layer_id = publish_json["services"][0]["serviceItemId"]

# Step 4: Verify the publishing status
print("Hosted Feature Layer ID:", hosted_feature_layer_id)

In [ ]:
from arcgis.gis import GIS
import getpass

gis = GIS(
    url="https://developers.arcgis.com",
    username="post4k",
    password=getpass.getpass("Enter password:"),
    use_gen_token=True,
)
